# Create_UserDic <a class="tocSkip">

In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime as dt
import MeCab

sys.path.append('../bin')
import myutil
import dataset
from dataset import Holomem, Hololive
import model
from model import Word2VecModel

prj_path = myutil.PROJECT_PATH
prj_path

PosixPath('/Users/nyu/gdrive/learn/hololi2vec')

<h1>目次<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#ユーザ辞書作成" data-toc-modified-id="ユーザ辞書作成-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>ユーザ辞書作成</a></span></li><li><span><a href="#ユーザ辞書テスト" data-toc-modified-id="ユーザ辞書テスト-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>ユーザ辞書テスト</a></span></li><li><span><a href="#分かち書き関数変更" data-toc-modified-id="分かち書き関数変更-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>分かち書き関数変更</a></span></li></ul></div>

# ユーザ辞書作成
- 分かち書き結果ファイルを中身を精査していると，ホロライブ特有の単語が正しく認識できていないことがわかった
- MeCabのカスタムユーザ辞書で対処するしかない

参考
- https://qiita.com/cove_ht/items/d743a9d757dd3097a6f6
- https://taku910.github.io/mecab/posid.html

**[辞書に追加する情報]**
- 名字・名前
- 特有の一人称
- ファンネーム
- コンビ・ユニット（一部メンバーのみ（wikiに記載が無いため））
- 呼び名（名前/あだ名+(ちゃん,先輩,様,氏,パイセン)除く）

備考
- 数字や記号を含む単語は除外（一部は数字や記号部分を省いて追加）
- 英大文字はすべて小文字に（正規化処理により小文字に統一されているため）
- 語録までは流石に作業量がデカすぎるので諦めている
- その他様々な理由で追加しなくて良いと自己判断したものは追加してない

**[情報収集源]**
- 所属タレント - ホロライブ（ホロライブ所属メンバーのみ）: https://www.hololive.tv/member
- ホロライブ非公式wiki（各メンバーページ）: https://seesaawiki.jp/hololivetv/

In [91]:
pd.set_option('display.max_rows', 400)

In [99]:
hololive_dic_path = Path(prj_path, 'data/hololive_dic.csv')
hololive_dic = pd.read_csv(hololive_dic_path, header=None)
hololive_dic

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,ときの,NaN,NaN,1,名詞,固有名詞,人名,姓,*,*,ときの,トキノ,トキノ
1,そら,NaN,NaN,1,名詞,固有名詞,人名,名,*,*,そら,ソラ,ソラ
2,そらとも,NaN,NaN,1,名詞,固有名詞,一般,*,*,*,そらとも,ソラトモ,ソラトモ
3,そらロボ,NaN,NaN,1,名詞,固有名詞,一般,*,*,*,そらロボ,ソラロボ,ソラロボ
4,そらみこ,NaN,NaN,1,名詞,固有名詞,一般,*,*,*,そらみこ,ソラミコ,ソラミコ
5,じゃあ敵だね,NaN,NaN,1,名詞,固有名詞,一般,*,*,*,じゃあ敵だね,ジャアテキダネ,ジャーテキダネ
6,ホロalice,NaN,NaN,1,名詞,固有名詞,一般,*,*,*,ホロalice,ホロアリス,ホロアリス
7,ロボ子さん,NaN,NaN,1,名詞,固有名詞,人名,一般,*,*,ロボ子さん,ロボコサン,ロボコサン
8,ろぼさー,NaN,NaN,1,名詞,固有名詞,一般,*,*,*,ろぼさー,ロボサー,ロボサー
9,ロボちゃん,NaN,NaN,1,名詞,固有名詞,人名,一般,*,*,ロボちゃん,ロボチャン,ロボチャン


全部手動で追加するしかなかった，地獄すぎ（色々抜けているかも...

In [106]:
hololive_dic.shape

(316, 13)

作成したcsvファイルを使い，ユーザ辞書を作成する

```
mkdir /usr/local/lib/mecab/dic/userdic

sudo /usr/local/Cellar/mecab/0.996/libexec/mecab/mecab-dict-index \
-d /usr/local/lib/mecab/dic/ipadic \
-u /usr/local/lib/mecab/dic/userdic/hololive.dic \
-f utf-8 \
-t utf-8 \
/Users/nyu/gdrive/learn/hololi2vec/data/hololive_dic.csv
```

homebrewを使ってインストールした影響で，`mecab-dict-index`は`/usr/local/Cellar/...`にある

`/usr/local/lib/mecab/dic/ipadict/dicrc`に以下追記

```
userdic = /usr/local/lib/mecab/dic/userdic/hololive.dic
```

# ユーザ辞書テスト

In [107]:
tete = 'シオンちゃんあてぃしのこと好きすぎ．あくあちゃんシオンのこと好きすぎ．'
tete

'シオンちゃんあてぃしのこと好きすぎ．あくあちゃんシオンのこと好きすぎ．'

NEologdを使ってもなお，「ぃしのこと」というようにおかしな分かち書きになる

In [108]:
neologd = MeCab.Tagger('-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
print(neologd.parse(tete))

シオン	シオン	シオン	名詞-固有名詞-人名-名		
ちゃん	チャン	ちゃん	名詞-接尾-人名		
あて	アテ	あて	名詞-接尾-一般		
ぃしのこと	ぃしのこと	ぃしのこと	名詞-一般		
好き	スキ	好き	名詞-接尾-形容動詞語幹		
すぎ	スギ	すぎ	名詞-接尾-副詞可能		
．	．	．	記号-句点		
あく	アク	あく	名詞-一般		
あちゃん	アチャン	あ~ちゃん	名詞-固有名詞-一般		
シオン	シオン	シオン	名詞-一般		
の	ノ	の	助詞-連体化		
こと	コト	こと	名詞-非自立-一般		
好き	スキ	好き	名詞-形容動詞語幹		
すぎ	スギ	すぎ	名詞-接尾-副詞可能		
．	．	．	記号-句点		
EOS



作成したユーザ辞書（ホロライブ辞書）を使うと...

In [109]:
neologd_hololive = MeCab.Tagger(
    '-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd -u /usr/local/lib/mecab/dic/userdic/hololive.dic')
print(neologd_hololive.parse(tete))

シオン	シオン	シオン	名詞-固有名詞-人名-名		
ちゃん	チャン	ちゃん	名詞-接尾-人名		
あてぃし	アティシ	あてぃし	名詞-代名詞-一般		
の	ノ	の	助詞-連体化		
こと	コト	こと	名詞-非自立-一般		
好き	スキ	好き	名詞-形容動詞語幹		
すぎ	スギ	すぎ	名詞-接尾-副詞可能		
．	．	．	記号-句点		
あくあ	アクア	あくあ	名詞-固有名詞-人名-名		
ちゃん	チャン	ちゃん	名詞-接尾-人名		
シオン	シオン	シオン	名詞-固有名詞-人名-名		
の	ノ	の	助詞-連体化		
こと	コト	こと	名詞-非自立-一般		
好き	スキ	好き	名詞-形容動詞語幹		
すぎ	スギ	すぎ	名詞-接尾-副詞可能		
．	．	．	記号-句点		
EOS



「あてぃし」や「あくあ」などが正確に分かち書きされている

# 分かち書き関数変更
- `Hololive`クラスの分かち書き結果ファイルを出力する関数である``__wakatigaki()``と，ツイートデータフレームからセッションデータ形式のリストを取得する``get_session_data()``において，MeCabの辞書に追加したユーザ辞書（ホロライブ辞書）も使うように変更した
- つまり，`MeCab.Tagger('-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd -u /usr/local/lib/mecab/dic/userdic/hololive.dic')`としてある

`Hololive`クラスの扱い方は`03_Prepare_Dataset`の5.2で述べた方法から変わっていないため，このノートで実際にコードを実行して説明することはしない